In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install emoji

# 交差検証テスト

In [ ]:
import pandas as pd
import emoji
#レビューリストとラベルリストを返す関数
def csv_to_list(csv_list : list):
  """
  csv_list => [タイトル, レビュー, カテゴリー, 星数, 参考になった数, 生成元ID]
  生成元IDはaug_csv系のみ
  """
  review_list,label_list = [],[]
  label_num = 8 #分類カテゴリ数
  each_label_num = {} #各label数のカウント
  #each_label_numのvalueを初期化
  for i in range(label_num):
    each_label_num[i+1] = 0
  for i in range(len(csv_list)):
    review = emoji.replace_emoji(csv_list[i][1])
    review_list.append(review)
    #csv_list[i][2]がintではなかったら
    if not isinstance(csv_list[i][2], int):
      str_category_list = csv_list[i][2].split(',')
    else:
      str_category_list = [csv_list[i][2]]
    try:
      category_list = [int(num) for num in str_category_list]
    except ValueError:
      print(i)
    onehot_lavel = []
    #空のマルチラベルリスト作成
    for i in range(label_num):
      onehot_lavel.append(0)
    for category in category_list:
      onehot_lavel[int(category)-1] = 1
      each_label_num[int(category)] =  each_label_num[int(category)] + 1
    label_list.append(onehot_lavel)
  return review_list, label_list, each_label_num

def get_aug_origin_ID(csv_list : list):
  aug_origin_ID = []
  for i in range(len(csv_list)):
    aug_origin_ID.append(csv_list[i][5])
  return aug_origin_ID

#シングルラベル分類用に上記のデータを整形
def multi_label_to_single_label(multi_label_list, target_label):
  single_label_list = []
  for i, _label in enumerate(multi_label_list):
    single_label_list.append(multi_label_list[i][target_label - 1])
  return single_label_list

#[0,1,1,0...0]の様なリストからtarget(0 or 1)であるリストのインデックスを返す関数
def get_label_index(label_list, target_label):
  label_index_list = []
  for i, _label in enumerate(label_list):
    if _label == target_label:
      label_index_list.append(i)
  return label_index_list


#インデックスリストを元にreviewsとlabelsの両方のリストから抽出したリストを返す関数
def get_review_label_from_index(label_index_list, reviews, labels):
  return [reviews[i] for i in label_index_list], [labels[i] for i in label_index_list]

# CSVファイルの読み込み
data = pd.read_csv('drive/MyDrive/review/review_ver20250108.csv')
review_label_list = data.values.tolist()
#オーグメンテーション用
data = pd.read_csv('drive/MyDrive/review/aug_review_label2-m1.csv')
aug_review_label_list = data.values.tolist()
"""
#シングルラベル用
data = pd.read_csv('drive/MyDrive/single_label_review.csv')
single_label_review_label_list = data.values.tolist()
"""
review_list,label_list, each_label_num = csv_to_list(review_label_list)
aug_review_list, aug_label_list, aug_each_label_num = csv_to_list(aug_review_label_list)
aug_origin_ID = get_aug_origin_ID(aug_review_label_list)
#single_label_review_list, single_label_label_list, single_label_each_label_num = csv_to_list(single_label_review_label_list)



In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
# ラベル2 (インデックス2) に着目　0〜7 -> 1〜8
n = 2
labels = np.array([row[n-1] for row in label_list])

# StratifiedKFold を使ってラベル2に基づいて均等に分割
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_indices = []

for train_idx, test_idx in skf.split(np.zeros(len(labels)), labels):
    fold_indices.append((train_idx, test_idx))

# 各foldごとのインデックスを表示
for i, (train_idx, test_idx) in enumerate(fold_indices):
    print(f"Fold {i + 1}:")
    print(f"  Train indices: {train_idx}\n Train_num : {len(train_idx)}")
    print(f"  Test indices: {test_idx}\n test_idx: {len(test_idx)}\n")


In [ ]:
# 各foldごとのインデックスを表示
#確認用リスト
confirm_list = []

for i, (train_idx, test_idx) in enumerate(fold_indices):

    # train_idxに対応するaug_origin_IDを探し、そのIDに一致するインデックスを取得
    selected_indexes = [i for i, origin_id in enumerate(aug_origin_ID) if origin_id in train_idx]
    # 上記のインデックスを使ってaug_review_listをフィルタリング
    train_aug_review_list = [aug_review_list[i] for i in selected_indexes]
    train_aug_label_list = [aug_label_list[i] for i in selected_indexes]

    test_selected_indexes = [i for i, origin_id in enumerate(aug_origin_ID) if origin_id in test_idx]


    # 結果表示
    print(f"{test_selected_indexes}")
    print(f"{aug_review_list[test_selected_indexes[0]]}")

    #print("Train Aug Review List:", train_aug_review_list)
    print(f"訓練用拡張データの数 : {len(train_aug_review_list)}")
    #print("Train Aug Label List:", train_aug_label_list)
    print(f"訓練用拡張データの数 : {len(train_aug_label_list)}\n")
    print(f"train_num : {len(train_idx)}")
    print(f"test_num : {len(test_idx)}\n")

[3, 4, 5, 15, 16, 17, 24, 25, 26, 27, 28, 29, 33, 34, 41, 42, 43, 53, 54, 55, 68, 69, 70, 71, 105, 106, 107, 108, 109, 110, 117, 118, 119, 120, 121, 122, 131, 132, 133, 150, 151, 152, 180, 181, 182, 192, 193, 227, 228, 229, 245, 246, 247]
　
もう一度購入したけれど、別のブランドになってしまっていて、比較してみると以前の方が風の強さがあったように感じる。個人的にはまあまあだと思う。
訓練用拡張データの数 : 204
訓練用拡張データの数 : 204

train_num : 612
test_num : 154

[18, 19, 20, 21, 22, 23, 65, 66, 67, 72, 73, 74, 90, 91, 92, 137, 138, 139, 141, 142, 143, 153, 154, 155, 162, 163, 164, 165, 166, 167, 183, 184, 185, 189, 190, 191, 203, 204, 205, 220, 221, 222, 223, 236, 237, 238, 239, 240, 241, 254, 255, 256]
　
充電が早く、バッテリーの持ちも良いです。この前Amazonで購入した同じタイプと比べると、非常に長持ちで充電もすぐに終わり、暑がりな私にはとても助かっています。冷却部分は冷たくは感じませんが、以前のモデルは風が当たらないことが多かったのに対し、こちらはフィット感が良く、猛暑の中では強めの設定でないと風を感じにくいです。
訓練用拡張データの数 : 205
訓練用拡張データの数 : 205

train_num : 613
test_num : 153

[6, 7, 8, 9, 10, 11, 12, 13, 14, 59, 60, 61, 62, 63, 64, 78, 79, 80, 84, 85, 86, 87, 88, 89, 96, 97, 98, 102, 103, 104, 114, 115, 116, 126, 127, 

In [ ]:
# train_idxに基づく処理
train_idx = 3  # 例: 生成元がインデックス3（review3）のデータを比較する

# aug_origin_IDから、train_idxに該当する拡張データのインデックスを取得
selected_indexes = [i for i, origin_id in enumerate(aug_origin_ID) if origin_id == train_idx]

# 生成元レビュー
origin_review = review_list[train_idx - 1]  # train_idxはインデックス+1なので、-1して取得

# 対応する拡張レビュー
corresponding_aug_reviews = [aug_review_list[i] for i in selected_indexes]

# 結果を表示
print(f"Train Index (Origin Review ID): {train_idx}")
print(f"Origin Review: {origin_review}")
print("Corresponding Augmented Reviews:")
for aug_review in corresponding_aug_reviews:
    print(f"  - {aug_review}")

Train Index (Origin Review ID): 3
Origin Review: ないと困る
5~6年前買った羽根型使ってたのが壊れ購入
断然こちらの方が、きっちり顔に風が来ていいです
内側の丸い部分は風が出ないとレビュー書いて下さってた意味使って理解てきた
そこが少し残念なのと
説明書に保証延長のLINE登録書いてあったけれど、問い合わせも登録も全く存在してない感じなので保証期間内に壊れたら心配
 
Corresponding Augmented Reviews:
  - 　
5～6年前に購入した羽根型が壊れたため、新たにこちらを購入しましたが、風がしっかり顔に届くので非常に満足しています。ただ、内側の丸い部分から風が出ないというレビューの意味が理解できたのは少し残念でした。また、説明書には保証延長のためのLINE登録について記載がありましたが、実際には問い合わせや登録ができないようで、保証期間内に故障するのが心配です。
  - 　
昔に購入した羽根型が壊れたため、こちらを新しく買ったのですが、風が顔に直接当たるのでとても良いです。しかし、内側の丸い部分から風が出ないというレビューを見て、その意味を理解できたのが少し残念です。また、説明書に記載されていた保証延長のLINE登録については、実際には存在しないようで、保証期間内に故障した場合が心配です。
  - 　
5～6年前の羽根型が壊れてしまったので、こちらを購入したところ、顔に風がしっかり当たるのがとても良いです。しかし、内側の丸い部分から風が出ないというレビューの意味を理解したのは少し残念でした。さらに、説明書には保証延長のLINE登録が書かれていましたが、問い合わせや登録が全くできないようで、保証期間内に壊れるかもしれないことが不安です。


# fbetaテスト

In [1]:
def calculate_f_beta(recall, precision, beta=0.5):
    """
    Calculate the F-beta score.

    Parameters:
        recall (float): Recall value (between 0 and 1).
        precision (float): Precision value (between 0 and 1).
        beta (float): Weighting factor for precision vs recall (default is 0.5 for F0.5 score).

    Returns:
        float: The F-beta score.
    """
    if recall == 0 and precision == 0:
        return 0.0  # To avoid division by zero

    beta_squared = beta ** 2
    f_beta = (1 + beta_squared) * (precision * recall) / (beta_squared * precision + recall)
    return f_beta

In [5]:
# Example usage:
recall = 0.33
precision = 0.55
f05_score = calculate_f_beta(recall, precision, beta=0.5)
print(f"F0.5 Score : {f05_score:.4f}")

F0.5 Score : 0.4853
